In [1]:
import importlib
import sys
sys.path.insert(0, '/cndd/fangming/CEMBA/snmcseq_dev')

from __init__ import *
from __init__jupyterlab import *
import datetime
from scipy import sparse
import collections
import itertools
import re
import fbpca
import pickle
import scipy.cluster.hierarchy as sch
from sklearn.metrics import euclidean_distances
import h5py

import snmcseq_utils
importlib.reload(snmcseq_utils)
import CEMBA_run_tsne
import CEMBA_clst_utils
import SCF_utils
importlib.reload(SCF_utils)

import seaborn as sns 
import matplotlib.pyplot as plt

- Explore cell numbers (data_freeze_neurons)
- Select cells
- SCF
- Post-SCF evaluation
- Get cells that are called IT in at least 2 methods


- remove IT-Car3 (SCF 3-3; RNA consensus 69; LIGER it-6)

In [2]:
f = '/cndd/fangming/CEMBA/data/MOp_all/results_final/cell_cluster_assignment_all_integrations.tsv'
meta_grand = pd.read_csv(f, sep='\t')
print(meta_grand.shape)
meta_grand.head()

(571703, 7)


/cndd/fangming/venvs/conda_dobro/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cell_id,SCF_round2,SCF_round3,RNA concensus,LIGER_level1,LIGER_level2,modality
0,snmcseq_gene_2C_M_0,1-1,1-1-1,NaN,0,it_8,DNA methylation
1,snmcseq_gene_2C_M_1,4-2,4-2-1,NaN,1,it_0,DNA methylation
2,snmcseq_gene_2C_M_100,4-1,4-1-1,NaN,1,it_0,DNA methylation
3,snmcseq_gene_2C_M_1000,4-2,4-2-1,NaN,1,it_0,DNA methylation
4,snmcseq_gene_2C_M_1001,4-1,4-1-1,NaN,1,it_0,DNA methylation


In [3]:
scf_clsts = ['1-1', '1-2', '3-1', '3-2', '4-1', '4-2']
cond1 = meta_grand['SCF_round2'].fillna('').isin(scf_clsts) 
meta_grand['isIT_SCF'] = 0
meta_grand.loc[cond1, 'isIT_SCF'] = 1

_x = meta_grand[cond1]
print(_x.shape)
_x.groupby(['modality', 'SCF_round2']).size().unstack().fillna(0).astype(int)

(237835, 8)


SCF_round2,1-1,1-2,3-1,3-2,4-1,4-2
modality,,,,,,
10X_cells_v2_AIBS,22021,11813,10442,8236,8123,2595
10X_cells_v3_AIBS,17226,7281,5535,4045,8415,2526
10X_nuclei_v3_AIBS,5237,1842,4778,3508,5280,1700
10X_nuclei_v3_Broad,14952,7120,9505,10221,13515,6996
DNA methylation,1078,793,915,836,893,1004
Open chromatin,10937,3896,3197,2525,9426,4560
SmartSeq_cells_AIBS,750,453,431,326,388,102
SmartSeq_nuclei_AIBS,496,239,448,517,621,92


In [4]:
f = '/cndd/fangming/CEMBA/data/MOp_all/results_final/cluster_annotation_rna_consensus.tsv'
df_annot_rna = pd.read_csv(f, sep='\t')
print(df_annot_rna.shape)

_x = df_annot_rna[df_annot_rna['annot'].str.contains('IT')]
print(_x)
_x = df_annot_rna[df_annot_rna['annot'].str.contains('IT_')]
print(_x)

(116, 3)
    cluster       annot    color
57       58   L2/3 IT_1  #00FF34
58       59   L2/3 IT_2  #07D945
59       60   L2/3 IT_3  #2EB934
60       61   L4/5 IT_1  #09CCC6
61       62   L4/5 IT_2  #52B8AA
62       63     L5 IT_1  #58D2C1
63       64     L5 IT_2  #4A9F93
64       65     L5 IT_3  #4EAC9C
65       66     L5 IT_4  #52B4B8
66       67     L6 IT_1  #B2AD23
67       68     L6 IT_2  #81791F
68       69  L6 IT Car3  #5100FF
    cluster      annot    color
57       58  L2/3 IT_1  #00FF34
58       59  L2/3 IT_2  #07D945
59       60  L2/3 IT_3  #2EB934
60       61  L4/5 IT_1  #09CCC6
61       62  L4/5 IT_2  #52B8AA
62       63    L5 IT_1  #58D2C1
63       64    L5 IT_2  #4A9F93
64       65    L5 IT_3  #4EAC9C
65       66    L5 IT_4  #52B4B8
66       67    L6 IT_1  #B2AD23
67       68    L6 IT_2  #81791F


In [5]:
rna_clsts = _x['cluster'].values

cond1 = meta_grand['RNA concensus'].fillna(-1).astype(int).isin(rna_clsts)
meta_grand['isIT_RNAconcensus'] = 0
meta_grand.loc[cond1, 'isIT_RNAconcensus'] = 1

_x = meta_grand[cond1]
print(_x.shape)
_x.groupby(['modality', 'RNA concensus']).size().unstack().fillna(0).astype(int)

(240989, 9)


RNA concensus,58.0,59.0,60.0,61.0,62.0,63.0,64.0,65.0,66.0,67.0,68.0
modality,,,,,,,,,,,
10X_cells_v2_AIBS,181,247,10750,19849,13472,7378,184,1026,3506,1987,4652
10X_cells_v3_AIBS,89,569,10382,15128,9240,3871,63,642,1713,1009,2382
10X_nuclei_v2_AIBS,701,1489,16308,7144,6552,5679,256,1502,3129,1099,2589
10X_nuclei_v3_AIBS,558,559,6606,3906,2284,2884,1,1080,1904,750,1289
10X_nuclei_v3_Broad,1410,2004,18685,11871,8232,7421,282,1394,4040,1408,2896
SmartSeq_cells_AIBS,0,1,484,644,562,280,28,26,110,53,262
SmartSeq_nuclei_AIBS,18,51,663,377,359,357,23,40,185,49,185


In [6]:
cond1 = ((meta_grand['LIGER_level2'].fillna('').str.contains('^it_')) 
        & (~meta_grand['LIGER_level2'].fillna('').str.contains('^it_6$')) # remove it_6
         )
         
meta_grand['isIT_LIGER'] = 0
meta_grand.loc[cond1, 'isIT_LIGER'] = 1

_x = meta_grand[cond1]
print(_x.shape)
_x.groupby(['modality', 'LIGER_level2']).size().unstack().fillna(0).astype(int)

(233234, 10)


LIGER_level2,it_0,it_1,it_10,it_2,it_3,it_4,it_5,it_7,it_8,it_9
modality,,,,,,,,,,
10X_cells_v2_AIBS,11068,20343,247,12832,9878,8614,4,155,19,93
10X_cells_v3_AIBS,10218,15967,5,8341,4907,4522,527,190,18,97
10X_nuclei_v3_AIBS,7372,4280,5,2084,4114,3424,50,98,19,87
10X_nuclei_v3_Broad,19136,13817,9,8041,9539,6935,1352,82,15,132
DNA methylation,1844,226,0,510,999,727,118,19,876,30
Open chromatin,13104,7157,11,3266,4324,4055,1261,691,286,388
SmartSeq_cells_AIBS,474,641,0,517,396,376,3,5,6,10
SmartSeq_nuclei_AIBS,711,418,0,321,418,383,7,8,3,9


In [7]:
meta_is = meta_grand[['cell_id', 'modality', 
                        'isIT_SCF', 'isIT_RNAconcensus', 'isIT_LIGER',
                       ]]
print(meta_is.shape)
meta_is.head()

(571703, 5)


,cell_id,modality,isIT_SCF,isIT_RNAconcensus,isIT_LIGER
0,snmcseq_gene_2C_M_0,DNA methylation,1,0,1
1,snmcseq_gene_2C_M_1,DNA methylation,1,0,1
2,snmcseq_gene_2C_M_100,DNA methylation,1,0,1
3,snmcseq_gene_2C_M_1000,DNA methylation,1,0,1
4,snmcseq_gene_2C_M_1001,DNA methylation,1,0,1


In [8]:
results = meta_is.groupby(['modality', 
                           'isIT_SCF', 
                           'isIT_RNAconcensus', 
                           'isIT_LIGER',
                         ]).size().reset_index().rename(columns={0: 'num'})
results['cond_code'] = (results['isIT_SCF'].astype(str)
                        + results['isIT_RNAconcensus'].astype(str)
                        + results['isIT_LIGER'].astype(str)
                       )
print(results.shape)
results.head()

(51, 6)


,modality,isIT_SCF,isIT_RNAconcensus,isIT_LIGER,num,cond_code
0,10X_cells_v2_AIBS,0,0,0,59360,000
1,10X_cells_v2_AIBS,0,0,1,17,001
2,10X_cells_v2_AIBS,0,1,0,12,010
3,10X_cells_v2_AIBS,0,1,1,22,011
4,10X_cells_v2_AIBS,1,0,0,13,100


In [9]:
decode_table = {
    '000': "None",
    '001': "LIGER only",
    '010': "RNA concensus only",
    '100': "SCF only",
    '110': "SCF and RNA",
    '101': "SCF and LIGER",
    '011': "RNA and LIGER",
    '111': "all",
}

datasets = [
    'SmartSeq_cells_AIBS', 
    '10X_cells_v3_AIBS', 
    '10X_cells_v2_AIBS', 
    'SmartSeq_nuclei_AIBS', 
    '10X_nuclei_v3_AIBS', 
    '10X_nuclei_v3_Broad', 
    '10X_nuclei_v2_AIBS', 
    'DNA methylation', 
    'Open chromatin', 
]

results_summary = (results.pivot(columns='cond_code', 
                                 index='modality', 
                                 values='num')
                            .fillna(0).astype(int)
                    )[['000', '100', '010', '001', 
                       '110', '101', '011', '111']].reindex(datasets).rename(columns=decode_table)
results_summary.columns.name = 'isPT?'
print(results_summary.shape)
results_summary

(9, 8)


isPT?,None,SCF only,RNA concensus only,LIGER only,SCF and RNA,SCF and LIGER,RNA and LIGER,all
modality,,,,,,,,
SmartSeq_cells_AIBS,3838,0,0,0,22,0,0,2428
10X_cells_v3_AIBS,26067,6,22,13,296,9,53,44717
10X_cells_v2_AIBS,59360,13,12,17,3,19,22,63195
SmartSeq_nuclei_AIBS,3754,109,3,1,27,0,0,2277
10X_nuclei_v3_AIBS,17789,551,11,2,282,3,19,21509
10X_nuclei_v3_Broad,97406,2678,11,4,585,7,8,59039
10X_nuclei_v2_AIBS,30077,0,46448,0,0,0,0,0
DNA methylation,3837,180,0,10,0,5339,0,0
Open chromatin,43508,1574,0,1576,0,32967,0,0


# Track information

In [10]:
f = '/cndd2/fangming/projects/miniatlas/results/it_candidates_strict_v2.tsv'

In [11]:
meta_is = meta_grand[meta_grand.filter(regex=r'^isIT', axis=1).sum(axis=1) >=2]
print(meta_is.shape)

meta_is = meta_is[~meta_is['SCF_round2'].isnull()]
print(meta_is.shape)
meta_is.head()

(232826, 10)
(232826, 10)


,cell_id,SCF_round2,SCF_round3,RNA concensus,LIGER_level1,LIGER_level2,modality,isIT_SCF,isIT_RNAconcensus,isIT_LIGER
0,snmcseq_gene_2C_M_0,1-1,1-1-1,NaN,0,it_8,DNA methylation,1,0,1
1,snmcseq_gene_2C_M_1,4-2,4-2-1,NaN,1,it_0,DNA methylation,1,0,1
2,snmcseq_gene_2C_M_100,4-1,4-1-1,NaN,1,it_0,DNA methylation,1,0,1
3,snmcseq_gene_2C_M_1000,4-2,4-2-1,NaN,1,it_0,DNA methylation,1,0,1
4,snmcseq_gene_2C_M_1001,4-1,4-1-1,NaN,1,it_0,DNA methylation,1,0,1


In [12]:
meta_is['modality'].value_counts()

10X_cells_v2_AIBS       63239
10X_nuclei_v3_Broad     59639
10X_cells_v3_AIBS       45075
Open chromatin          32967
10X_nuclei_v3_AIBS      21813
DNA methylation          5339
SmartSeq_cells_AIBS      2450
SmartSeq_nuclei_AIBS     2304
Name: modality, dtype: int64

In [13]:
meta_is.to_csv(f, sep='\t', header=True, index=False)